<a href="https://colab.research.google.com/github/carldegs/EE-286/blob/master/EE286_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Genre Classifier

Narz Marbeth David 2005-09925
Carl Justin de Guia 2011-06521

## Introduction
// TODO

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.regularizers import l2

import warnings
warnings.filterwarnings('ignore')



## Dataset

The dataset used for training this genre classifier is from the Million Song Dataset [2]. To access the dataset, an AWS instance was started and mounted with the Million Song Dataset. The dataset contains features derived by the Echo Nest API currently documented in Spotify's repositories [4]. The fields will be detailed further in the Feature Extraction section. Additionally, a genre ground truth was provided in [3] to assign genre labels to each song in the dataset.

In [5]:
# TODO: Import dataset and show graphs and stuff
import numpy as np
import os.path

## The following shows how a track is organized in the dataset 
dataset_path = r"/mnt/snap/MillionSongDatabase/data"

def get_track_path(trackId, dataset_path):
    track_path = os.path.join(dataset_path, trackId[2],trackId[3], trackId[4]);
    return track_path


In [6]:
## Read the genre mapping table
genre_map = pd.read_csv('msd-MAGD-genreAssignment.cls', comment='#', sep='\t', header=None, usecols=[0,1], names=['TrackId', 'Genre'])


## Feature Extraction

The following features are extracted from the training set: 

- Tempo
- Average Beat Period
- Within 30 seconds of the song:
    - Timbre Coefficients
    - Pitch Classes
    - Intensity

In [7]:
## This shows what features are available per track 

def print_track_details(trackId):
    trackPath = get_track_path(trackId, dataset_path)
    filename = os.path.join(trackPath, trackId + ".h5")
    with hdf5_getters.open_h5_file_read(filename) as h5:
        numSongs = hdf5_getters.get_num_songs(h5)
        getters = list(filter(lambda x: x[:4] == 'get_', hdf5_getters.__dict__.keys()))
        getters.remove("get_num_songs") # special case
        for getter in getters:
            res = hdf5_getters.__getattribute__(getter)(h5,0)
            if res.__class__.__name__ == 'ndarray':
                print(getter[4:]+": shape =",res.shape)
            else:
                print(getter[4:]+":",res)


## Timbre 

The timbre feature is a vector that includes 12 unbounded values roughly centered around 0. Those values are high level abstractions of the spectral surface, ordered by degree of importance. The actual timbre of the segment is best described as a linear combination of these 12 basis functions weighted by the coefficient values: 

timbre = c1 x b1 + c2 x b2 + … + c12 x b12, 

where c1 to c12 represent the 12 coefficients and b1 to b12 the 12 basis functions.[4]

In [180]:
## Construct training set headers
headers = ["trackId", "tempo"]
segment_timbre_cols = []
for i in range(20):
    for j in range(12):
        ## Timbre column := "seg_timbre_" + seg_id + "_" + t_coeff_id
        segment_timbre_col = "seg_timbre_%d_%d"%(i,j)
        segment_timbre_cols.append(segment_timbre_col)
        headers.append(segment_timbre_col)
segment_pitches_cols = []        
for i in range(20):
    for j in range(12):
        ## Timbre column := "seg_pitches_" + seg_id + "_" + pitch_class_id
        segment_pitches_col = "seg_pitches_%d_%d"%(i, j)
        segment_pitches_cols.append(segment_pitches_col)
        headers.append(segment_pitches_col)
segment_loudness_cols = []    
for i in range(20):
    segment_loudness_col = "seg_loudness_%d"%(i)
    segment_loudness_cols.append(segment_loudness_col)
    headers.append(segment_loudness_col)
headers.append("ave_beat_period")    
headers.append("label")
df = pd.DataFrame(columns=headers)
df
len(segment_timbre_cols)

240

In [39]:
trackId='TRAAAAK128F9318786'
filename = os.path.join(get_track_path(trackId, dataset_path), trackId + ".h5")
track = Track(trackId, filename)
track.setGenre('Pop')
track.extractFeatures()


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
from Track import Track

## Generate training set per Genre
genre_set = ['Pop_Rock', 
             'RnB',
             'Classical',
             'Blues',
             'Electronic',
             'Latin',
             'Jazz',
             'Rap',
             'Religious']

## Write CSV headers of training sets
#for genre in genre_set:
#    with open(genre + '.csv', 'w') as f:
#        writer = csv.DictWriter(f, fieldnames=headers)
#        writer.writeheader()
for genre in genre_set:
    genre_data = genre_map[genre_map['Genre'] == genre]
    i=0
    df=pd.DataFrame(columns=headers)
    for index, row in genre_data.iterrows():
        if (i > 3000):
            break
        i = i+1
        ## Retrieve track metadata from dataset
        trackId = row['TrackId']
        filename = os.path.join(get_track_path(trackId, dataset_path), trackId + ".h5")
        track = Track(trackId, filename)
        track.setGenre(genre)
        track.extractFeatures()

        data = np.concatenate([[track.trackId], [track.tempo], track.segments_timbre, track.segments_pitches, track.segments_loudness_max, [track.beats_average_period], [track.genre]])
        df.loc[0] = data
        df['trackId']
    df.to_csv(genre + '.csv', index=None, header=True);

In [199]:
# TODO: Do the extraction and show ouput data
# TODO (Minor??): Save extracted data to a .csv file to remove necessity to run feature extraction code multiple times.

## Training the Model

Using the features extracted, a 3-layer Multilayer perceptron (MLP) is trained to determine a song's genre.

![image.png](https://res.cloudinary.com/practicaldev/image/fetch/s--5hmoQpw5--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_880/https://cdn-images-1.medium.com/max/720/1%2AvWRGnasRs2zo3GhTHlmIfg.jpeg)
<center>A visualization of an MLP</center>

A MLP is a class of feeedforward artificial neural network that utilizes backpropagation, a supervised learning technique used during training. Basically, the MLP will receive the features in the input layer and will return a set of probabilities for each genre in the output layer.

To make the training more effective, we pre-process our data further.
First we separate the features from the labels, then we standardized the values of the features to train our model faster. Standardization transforms the distribution of each feature to have a mean value of 0 and standard deviation of 1. Meanwhile, the output is converted into integers then transformed into a one-hot vector. One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction. We then split the inputs and the output labels into a training and test set.

Afterwards, some hyperparameters are set which will be used by the model during training.


In [ ]:
## Read extracted feature data and split into input and output.
data = pd.read_csv('./test_data.csv')

X = data.drop(['trackId', 'label'], axis=1)
y = data['label']

num_labels = len(np.unique(y))
input_size = X.shape[1]

# Standardize floats
scaler = StandardScaler()
X = scaler.fit_transform(np.array(X, dtype = float))

# convert labels to one-hot vector
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

# Split into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y)

# hyperparameters
batch_size = 128
hidden_units = 256
dropout = 0.4
epochs=100

# Model is a 3-layer MLP with ReLU and dropout after each layer
model = Sequential()

model.add(Dense(hidden_units,input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))

model.add(Dense(hidden_units))
model.add(Activation('relu'))
model.add(Dropout(dropout))

model.add(Dense(num_labels))

# Output
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

# Train the network
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

# validation
loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

# Results

We can measure how correct the model is by looking into the model loss and it's accuracy. Loss is a number indicating how bad the model's prediction is on a given sample. The loss should decrease as training continues. But too much training will overfit our model and increase the loss. Accuracy meanwhile determines how correct the predictions are. This is determined by using the model with a different set of inputs, the test data we separated earlier, and comparing the model's output with the actual answer. This should increase as training continues but again when the model is being trained for too long, the model will overfit and using the data from the test set will yield a lower accuracy.

For comparison, we use Pandey's [1] MLP which yields an accuracy of about 65%.

Just by changing the hyperparameters and adding some regularizers and other techniques to optimize the model, our model's accuracy increased to at maximum, **70.8%**.

And after adding the **INSERT FEATURE HERE** and **INSERT OTHER FEATURE HERE**, we further increased the accuracy to **INSERT BETTER ACCURACY** here.

In [ ]:
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

# Summarize history for loss
plt.plot(history.history['loss'])
plt.plot(loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

## Reference

[1] Pandey, P. (2018, December 19). Music Genre Classification with Python. Retrieved from https://towardsdatascience.com/music-genre-classification-with-python-c714d032f0d8.

[2] Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, and Paul Lamere. The Million Song Dataset. In Proceedings of the 12th International Society for Music Information Retrieval Conference (ISMIR 2011), 2011.

[3] Information and Software Engineering Group (IFS), Institute of Software Technology and Interactive Systems (ISIS),
Vienna University of Technology. MSD Allmusic Genre Dataset (MAGD). Retrieved from http://www.ifs.tuwien.ac.at/mir/msd/download.html

[4] Retrieved from https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/#timbre